# Simular Basics
This example demonstrates how to use Simular to deploy and interact with an Ethereum smart contract.

**What we'll do:**
- Create an instance of the EVM
- Create an account to deploy the contract and transact
- Deploy and interact with the contract

We'll use a very simple contract: `Counter.sol` (source is available in this directory). It's sole purpose in life is to keep track of an incrementing value. The contract has 2 methods that can be called:  
- `increment()`   adds one to the current state of the counter
- `number() returns uint256` returns the current value of the counter

You can find more examples and details on the API here: https://simular.readthedocs.io/en/latest/

## Setup

First, we'll import the EVM and a few helper functions from simular. Then, we'll load
the ABI and bytecode information from the compiled contract.  `Counter.json` is the output
from compiling our contract in [foundry](https://book.getfoundry.sh/). It contains 2 things we need: the ABI, or interface to the contract's methods, and the compiled bytecode of the contract.

In [1]:
from simular import PyEvm, create_account, contract_from_raw_abi

# Note: we read as a string. Simular handles parsing into JSON
with open('./Counter.json') as f:
    rawabi = f.read()

## Create the EVM and Account
We always need an instance of the EVM. And for this example, we'll need a single Ethereum account
to deploy the contract.

In [2]:
evm = PyEvm()
deployer = create_account(evm)
print(f"deployers address: {deployer}")

deployers address: 0xe1ef4a53a7495843670a29d2d0da243a2554b0d6


## Contract
Next, we create an instance of the Contract object.  It requires the `rawabi` information (from Counter.json above) and an instance of the EVM.

In [3]:
counter = contract_from_raw_abi(evm, rawabi)

Deploy the contract to the EVM. Returning the contract's address

In [4]:
contract_address = counter.deploy(caller=deployer)
print(f"contract address: {contract_address}")

contract address: 0x6A786129D27071341f5A62b1F64c8316E8c96d7e


## Calling the contract.

Simular will automagically add the smart contract functions from the ABI to the instance of the contract object as attributes.  For example, the counter contract we created has 2 methods: `increment` and `number`.  If you reference the attributes, i.e. `counter.increment`, you'll see this returns a *Function*.  To actually invoke the methods we need to append one of the following:
- `transact(..)`:  this makes a **write** call to the contract, or
- `call(..)`: this makes a **read** call to the contract.

Both `transact` and `call` take arguments that are expected by the corresponding smart contract method.  If you get an error that says something like: *the arguments don't match the expected input...* that means you did not pass the correct input to the method as defined in the ABI.

Let's call `number`.  This is a read-only method that take no input arguments and returns the current 'count' state in the contract.  

In [5]:
print(counter.number.call())

0


Next we'll make some `transact` (write) calls to the contract using `increment`. Note, write calls use 
`transact` and at a minimum **require** the caller's address.  `caller` translates to `msg.sender` in the contract.

In [6]:
for i in range(5):
    counter.increment.transact(caller=deployer)

Let's check the value of number again...

In [7]:
print(counter.number.call())

5
